<a href="https://colab.research.google.com/github/axel-sirota/pre-trained-nlp-models/blob/main/module2/PreTrainedNLP_Mod2Demo1_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning GloVe Vectors as Embeddings

## Initialization and Data Preprocessing

In [1]:
!nvidia-smi

Tue Jan  9 01:10:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install --upgrade  textblob keras-nlp keras-preprocessing np-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 91.5 MB/s eta 0:00:00
  Created wheel for np-utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=53b8d945fd56e748ac3a63999f7c69040b5dc7a77bd47b009c993c91e33b5382
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np-utils


In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

batch_size = 64
embedding_dim = 100
corpus_size=100
epochs=10

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)


set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2024-01-09 01:10:39--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2024-01-09 01:10:39--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce90a524d48c2bb4a51b32b352e.dl.dropboxusercontent.com/cd/0/inline/CK_pyrbnlQMEgGSnxs1D6V0eoCKKj5kmpzt37Znb5SSbhGGplMEaVqtKI_sW_fy8ceMASQUQTQ1c8U91A2ediKSvwgy_mL922-tZ7YDlguTzUuqPxX2lhR61MULNSDHwVLjHGtF1bl2dsPXsU8Vhf325/file# [following]
--2024-01-09 01:10:39--  https://uce90a524d48c2bb4a51b32b352e.dl.dropboxusercontent.com/cd/0/inline/CK_pyrbnlQMEgGSnxs1D6V0eoCKKj5kmpzt37Znb5SSbhGGplMEaVqtKI_sW_fy8ceMASQUQTQ1c8U91A2ediKSvwgy_mL922-tZ7YDlgu

In [6]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [7]:
#corpus = [sentence for sentence in X.values if type(sentence) == str and len(TextBlob(sentence).words) > 3]

In [8]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [9]:
corpus = X.values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
tokenized_corpus = tokenizer.texts_to_sequences(corpus)
vocab_size = len(tokenizer.word_index) + 1

In [10]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')

First 5 corpus items are ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'
 'I have no idea why some people give bad reviews about this place. It g

## Model Construction

In [11]:
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17193 words (2760 misses)


In [12]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=4, embeddings_initializer=Constant(embedding_matrix), trainable=True))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,)))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            1995400   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 19954)             2015354   
                                                                 
Total params: 4010754 (15.30 MB)
Trainable params: 4010754 (15.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Dataset Preparation

In [14]:
def create_context_target_pairs(texts, vocab_size=vocab_size, context_size=2):
    data = []
    for text in texts:
        tokens = text.split()
        for i, word in enumerate(tokens):
            start = max(0, i - context_size)
            end = min(len(tokens), i + context_size + 1)
            sequence = [tokens[j] for j in range(start, end) if j != i]
            try:
              sequence = tf.squeeze(tf.constant(tokenizer.texts_to_sequences(sequence)))
              context = tf.keras.utils.pad_sequences([sequence], maxlen=4, padding='post')
            except Exception:
              continue
            target = tokenizer.word_index[word] if word in tokenizer.word_index else 0
            context_indices = np.append(context, target)
            if len(context_indices) != 5:
              continue
            data.append(tf.constant(context_indices))
    data = tf.stack(data)
    X = data[:,:4]
    y = data[:,4]
    y = tf.one_hot(y, vocab_size)
    return X,y

In [15]:
create_context_target_pairs(corpus[0:4])

(<tf.Tensor: shape=(542, 4), dtype=int64, numpy=
 array([[447, 202,   0,   0],
        [ 12, 202,  35,   0],
        [ 12, 447,  35,  41],
        ...,
        [ 93, 818,   4, 284],
        [818,  94, 284,  11],
        [ 94,   4,  11, 517]])>,
 <tf.Tensor: shape=(542, 19954), dtype=float32, numpy=
 array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

## Model Training

In [16]:
tf.config.run_functions_eagerly(True)
x , y = create_context_target_pairs(texts=corpus[0:corpus_size])
history = model.fit(x, y, epochs=epochs, validation_split=0.2, batch_size=batch_size, workers=5)
print(history)

Epoch 1/10
137/137 [==============================] - 6s 28ms/step - loss: 7.9279 - val_loss: 6.1574
Epoch 2/10
137/137 [==============================] - 4s 26ms/step - loss: 5.5449 - val_loss: 5.8278
Epoch 3/10
137/137 [==============================] - 4s 26ms/step - loss: 5.2632 - val_loss: 5.7746
Epoch 4/10
137/137 [==============================] - 4s 26ms/step - loss: 5.0916 - val_loss: 5.7367
Epoch 5/10
137/137 [==============================] - 3s 25ms/step - loss: 4.9418 - val_loss: 5.6997
Epoch 6/10
137/137 [==============================] - 4s 26ms/step - loss: 4.8032 - val_loss: 5.6738
Epoch 7/10
137/137 [==============================] - 3s 25ms/step - loss: 4.6685 - val_loss: 5.6584
Epoch 8/10
137/137 [==============================] - 4s 26ms/step - loss: 4.5337 - val_loss: 5.6499
Epoch 9/10
137/137 [==============================] - 4s 26ms/step - loss: 4.4010 - val_loss: 5.6346
Epoch 10/10
137/137 [==============================] - 4s 26ms/step - loss: 4.2668 - val_lo

## Validating

In [17]:
with open('./vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
    vectors = model.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))

In [18]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)


In [19]:
w2v.most_similar(positive=['pizza'])

[('sandwich', 0.7827951908111572),
 ('taco', 0.7409960031509399),
 ('burger', 0.7357674241065979),
 ('pie', 0.7292145490646362),
 ('pepperoni', 0.7228743433952332),
 ('bread', 0.7175434231758118),
 ('chicken', 0.7148180603981018),
 ('sandwiches', 0.7052547335624695),
 ('fried', 0.700016438961029),
 ('restaurant', 0.6883482933044434)]